In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *

In [3]:
# the raw nc files for VZA were downloaded for 15 days, not per month as LST
# for easier managing, files will be merged here

files = sorted(getFilelist('/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw', '.nc'))
year = 2019
outPath = f'/data/Aldhani/eoagritwin/et/Sentinel3/VAA/monthly_tiff_values/{year}/'
os.makedirs(outPath, exist_ok=True) 
    

In [4]:
# get a subset of files for that year
yearFiles = [file for file in files if int(file.split('/')[-1].split('_')[-1][0:4]) == year]

In [5]:
for month in [f'{i:02d}' for i in range(1,13)]:
    print([yF for yF in yearFiles if month == yF.split('-')[1]])

['/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-01-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-01-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-02-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-02-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-03-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-03-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-04-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-04-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-05-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-05-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-06-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-06-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-07-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VAA/raw/Germany_2019-07-15.nc']

In [5]:
for month in [f'{i:02d}' for i in range(1,13)]:
    if month == '02':
        break
    monthL = []
    files = [yF for yF in yearFiles if month == yF.split('-')[1]]
    for file in files:
        monthL.append(getDataFromNC_VAA(file))
    block = np.dstack(monthL)

In [6]:
for month in [f'{i:02d}' for i in range(1,13)]:
    files = [yF for yF in yearFiles if month == yF.split('-')[1]]
    monthL = []
    bnames = []
    for file in files:
        monthL.append(getDataFromNC_VAA(file))
        accDateTimes = getAllDatesS3(file)
        for l in range(len(accDateTimes)):
            bnames.append(str(accDateTimes[l].astype('datetime64[s]')))

    block = np.dstack(monthL)
    # block2 = np.where(block > 45, 0, 1)
    # block2[np.isnan(block)] = -5
    # block3 = block2.astype(np.int8)

    exportNCarrayDerivatesInt(file, outPath, f'VAA_{year}_{month}.tif', bnames, block, datType=gdal.GDT_Float32, numberOfBands=block.shape[2])

/data/Aldhani/users/potzschf/conda/envs/workhorse/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
